# Fetch YT Info

Use the YouTube Data API to fetch info for the videos in a given playlist. Results can be downloaded as external files.

- [YouTube Data API Overview](https://developers.google.com/youtube/v3/getting-started)

In [2]:
API_KEY = input("Enter your YouTube Data API key: ")

Enter the YouTube playlist ID (the part after 'list='): PLMBTl5yXyrGRl2_kwa3tB2imqkb08_KvD


In [6]:
import requests
import pandas as pd

# Get all video IDs in the playlist
def get_video_ids(playlist_id, api_key):
    video_ids = []
    base_url = "https://www.googleapis.com/youtube/v3/playlistItems"
    params = {
        "part": "contentDetails",
        "playlistId": playlist_id,
        "maxResults": 50,
        "key": api_key
    }

    while True:
        res = requests.get(base_url, params=params).json()
        for item in res.get("items", []):
            video_ids.append(item["contentDetails"]["videoId"])
        next_token = res.get("nextPageToken")
        if not next_token:
            break
        params["pageToken"] = next_token

    return video_ids

# Get stats (views, likes) for each video
import requests
def get_video_stats(video_ids, api_key):
    stats = []
    base_url = "https://www.googleapis.com/youtube/v3/videos"
    for i in range(0, len(video_ids), 50):  # max 50 IDs per request
        ids_slice = video_ids[i:i+50]
        params = {
            "part": "statistics,snippet",
            "id": ",".join(ids_slice),
            "key": api_key
        }
        res = requests.get(base_url, params=params).json()
        for item in res.get("items", []):
            stats.append({
                "video_id": item["id"],
                "title": item["snippet"]["title"],
                "views": int(item["statistics"].get("viewCount", 0)),
                "likes": int(item["statistics"].get("likeCount", 0)),
                "upload_date": item["snippet"].get("publishedAt", "")
            })
    return stats

## Fetch data from playlist

In [19]:
PLAYLIST_ID = input("Enter the YouTube playlist ID (the part after 'list='): ")

Enter the YouTube playlist ID (the part after 'list='): PLMBTl5yXyrGSa67JeYb6ctt1gCvL9BdEX


In [20]:
# Run and show results (pings API)
video_ids = get_video_ids(PLAYLIST_ID, API_KEY)
video_stats = get_video_stats(video_ids, API_KEY)

In [26]:
video_stats = pd.read_csv("youtube_data_greatest_hits.csv")

In [27]:
# Format
df = pd.DataFrame(video_stats)
df['fetch_date'] = pd.Timestamp.now(tz='UTC').replace(microsecond=0).isoformat().replace('+00:00', 'Z')
df = df.sort_values(by="upload_date", ascending=True).reset_index(drop=True)

In [29]:
# export
df.to_csv("youtube_data_greatest.csv", index=False)